# Solution for UniveRSAlity

_There was no particular inspiration for this challenge -- I just wanted a -lity word with the letters RSA in it, and so  UniveRSAlity was born which clearly meant I had to do something with universal constants._

In [1]:
from pwn import *
from Crypto.Util.number import bytes_to_long
from sympy.ntheory import discrete_log, isprime
from sympy.ntheory.factor_ import smoothness
from sympy.ntheory.modular import crt

This is a typical application of the Pohlig-Hellman algorithm. The idea is to find sufficiently smooth primes $p$ and $q$ so that we can quickly solve the discrete log problem. According to the Canfield-Erdös-Pomerance Theorem, 1 in 256 numbers under $2^{128}$ will be $2^{32}$-smooth, so we can randomly generate random numbers until we get one that satisfies our requirements.

For extra elegance, we look for primes that have the same digits as $\pi$ and $e$ in all but the last 5 digits, and sort these by smoothness. The density of primes is roughly $\frac{1}{\log\left(2^{128}\right)}$ in this interval, so we can expect $\frac{10^5}{256\log\left(2^{128}\right)} \approx 4.4$ candidates for each prime.

We really only need to find one such pair of primes, but it's useful to have more candidates anyway, just in case some pairs of $(p, q)$ don't have a discrete log for our particular ciphertext.

In [2]:
def get_smooth_values_in_interval(start, len):
    from tqdm import trange
    result = []
    for n in trange(start, start + len):
        if isprime(n):
            s = smoothness(n-1)[0]
            if s < 2**32:
                result.append((s, n))
    return sorted(result)

In [3]:
SMOOTH_PI = get_smooth_values_in_interval(314159265358979323846264338327950200000, 10**5)
SMOOTH_E  = get_smooth_values_in_interval(271828182845904523536028747135266200000, 10**5)

100%|██████████| 100000/100000 [21:38<00:00, 77.04it/s] 


Took us quite a while to get through the intervals, but fortunately it's a one-time procedure. We can see from the next segment of code that we have 4 candidates for $p$ and 6 candidates for $q$.

In [4]:
SMOOTH_PI, SMOOTH_E

([(34494743, 314159265358979323846264338327950200873),
  (71743787, 314159265358979323846264338327950277143),
  (932051051, 314159265358979323846264338327950248901),
  (2067633571, 314159265358979323846264338327950200481)],
 [(52824803, 271828182845904523536028747135266272641),
  (269793047, 271828182845904523536028747135266263831),
  (289913639, 271828182845904523536028747135266245663),
  (322842217, 271828182845904523536028747135266282137),
  (922399679, 271828182845904523536028747135266289711),
  (2914586413, 271828182845904523536028747135266225409)])

Now, the last remaining ingredient of the challenge is to make `'flag'` appear in the JSON, which can be done by making it the key of a single-digit integer. The parameters are constructed so that the plaintext message must be no larger than 32 bytes. Since the token has length 11, this gives us just enough space to fit a single-value integer for `flag`, as in:

```
{"token":"###########","flag":#}
0        1         2         3
12345678901234567890123456789012
```

Incidentally, this also allows us to increase our universe of possible plaintext messages by 10x.

In [5]:
def get_answers(token):
    m1 = bytes_to_long(f'{{"token": "{token}"}}'.encode())
    for i in range(10):
        m2 = bytes_to_long(f'{{"token":"{token}","flag":{i}}}'.encode())
        for _, p in SMOOTH_PI:
            for _, q in SMOOTH_E:
                c = pow(m1, 65537, p * q)
                try:
                    d = crt([p-1,q-1],[discrete_log(p,m2,c),discrete_log(q,m2,c)])[0]
                    yield (p, q, d)
                except (ValueError,TypeError):
                    pass

Essentially, we've written `get_answers()` as a function that can output all possible (p,q,d) tuples in our search space, but we really only need one such tuple for our exploit. Here's the final exploit:

In [6]:
with remote('fun.chall.seetf.sg', 30002) as sh:
    sh.recvuntil(b'"')
    token = sh.recvuntil(b'"', True).decode()
    print(f'The token is "{token}".')

    ans = next(get_answers(token))
    print(f'Using (p,q,d)={ans}')

    sh.sendlines(str(a).encode() for a in ans)
    sh.recvuntil(b'passed!\n')
    print(sh.recvline(False))

[x] Opening connection to fun.chall.seetf.sg on port 30002
[x] Opening connection to fun.chall.seetf.sg on port 30002: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 30002: Done
The token is "EyNuA9EN4vg".
Using (p,q,d)=(314159265358979323846264338327950200873, 271828182845904523536028747135266263831, 20050771741598258276360761461642940690676782474875218942468459772177709155161)
b'SEE{pohlig-hellman_easy_as_pie_db01d3f24beda43e}'
[*] Closed connection to fun.chall.seetf.sg port 30002
